In [ ]:
# | default_exp _components.integration_test_generator

In [ ]:
# | export


from typing import *
import shutil
from pathlib import Path
import os
import toml
import re
from tempfile import TemporaryDirectory
from contextlib import contextmanager
from collections import defaultdict
import subprocess  # nosec: B404: Consider possible security implications associated with the subprocess module.

from yaspin import yaspin

from faststream_gen._components.logger import get_logger

from faststream_gen._code_generator.prompts import REQUIREMENTS_GENERATION_PROMPT
from faststream_gen._code_generator.constants import (
    APPLICATION_FILE_PATH,
    TEST_FILE_PATH,
    STEP_LOG_DIR_NAMES,
    TOML_FILE_NAME,
    OpenAIModel,
)

from faststream_gen._code_generator.chat import CustomAIChat, ValidateAndFixResponse

from faststream_gen._code_generator.helper import (
    write_file_contents,
    read_file_contents,
    set_cwd,
    mock_openai_create,
    retry_on_error,
)

In [ ]:


import pytest

from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

create_venv_and_run_tests_bash_script = """
#!/bin/bash

# get the project directory from command line argument
project_dir=$1

# get the venv directory from command line argument
venv_dir=$2

# create a new venv in the specified directory
python3 -m venv $venv_dir/my_venv > /dev/null 2>&1

# activate the venv
source $venv_dir/my_venv/bin/activate

# navigate to the project directory
cd $project_dir

# install the python project inside the venv
pip install .['dev'] > /dev/null 2>&1

# run pytest and capture output
pytest_output=$(pytest --tb=short)

# capture pytest exit code
pytest_exit_code=$?

# print the pytest output
echo "pytest_output_start:$pytest_output:pytest_output_end"

# print the pytest exit code
echo "pytest_exit_code:$pytest_exit_code"

# deactivate the venv
deactivate
"""

In [ ]:
# | export


def _setup_venv_and_run_tests(output_path: str) -> List[str]:
    output_path_resolved = Path(output_path).resolve()
    with TemporaryDirectory() as d:
        bash_file = Path(d) / "run_tests.sh"
        write_file_contents(str(bash_file), create_venv_and_run_tests_bash_script)
        with set_cwd(d):
            # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
            p = subprocess.run( # nosec: B602, B603, B607 subprocess call - check for execution of untrusted input.
                ["bash", "run_tests.sh", output_path_resolved, Path(d).resolve()],
                capture_output=True,
                text=True,
            )
            
        # Extract exit code
        exit_code = int(re.search('pytest_exit_code:(\d+)', p.stdout).group(1)) # type: ignore
        if exit_code !=0:
            # Extract pytest output
            pytest_output: str = re.search('pytest_output_start:(.*):pytest_output_end', p.stdout, re.DOTALL).group(1).strip() # type: ignore
            return [pytest_output]
        
        return []    

In [ ]:
fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
def test_always_pass():
    assert True
"""

fixture_pytoml_file = """
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "app"
version = "0.0.1"
dependencies = [ "faststream[kafka, docs]>=0.1.5", "pydantic", "ssl", "requests",]

[project.optional-dependencies]
lint = [ "black==23.9.1",]
static-analysis = [ "types-PyYAML",]
testing = [ "faststream[kafka, testing]>=0.1.5", "pytest",]
dev = [ "app[lint,static-analysis,testing]",]

[tool.pytest.ini_options]
filterwarnings = [ "ignore::DeprecationWarning",]
"""

with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_pytoml_file)
    
    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()

    actual = _setup_venv_and_run_tests(d)
    print(actual)
    assert actual == []

[]


In [ ]:
fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
import unknown_package
def test_always_fails():
    assert False
"""

fixture_pytoml_file = """
requires = ["hatchling"]
build-backend = "hatchling.build"

[project]
name = "app"
version = "0.0.1"
dependencies = [ "faststream[kafka, docs]>=0.1.5", "pydantic", "ssl", "requests",]

[project.optional-dependencies]
lint = [ "black==23.9.1",]
static-analysis = [ "types-PyYAML",]
testing = [ "faststream[kafka, testing]>=0.1.5", "pytest",]
dev = [ "app[lint,static-analysis,testing]",]

[tool.pytest.ini_options]
filterwarnings = [ "ignore::DeprecationWarning",]
"""

with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_pytoml_file)
    
    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()

    actual = _setup_venv_and_run_tests(d)
    print(actual[0])
    assert actual != []
    print("OK")

============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: /tmp/tmplojftuue
configfile: pyproject.toml
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 0 items / 1 error

==================================== ERRORS ====================================
__________________ ERROR collecting tests/test_application.py __________________
ImportError while importing test module '/tmp/tmplojftuue/tests/test_application.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
/usr/lib/python3.11/importlib/__init__.py:126: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
tests/test_application.py:2: in <module>
    import unknown_package
E   ModuleNotFoundError: No module named 'unknown_package'
=========================== short test summary info ============================
ERROR tests/test_application.py
!!!!!!!!!!!!!!!!!!!

In [ ]:
# | export

def _stript(s: str) -> str:
    return s.strip().strip('"')


def _split_app_and_test_req(response: str) -> Tuple[str, str]:

    app_req, test_req = response.split("==== APP REQUIREMENT ====")[1].split(
        "==== TEST REQUIREMENT ===="
    )
    return _stript(app_req), _stript(test_req)

In [ ]:
response = """
==== APP REQUIREMENT ====
"pydantic, ssl, requests"

==== TEST REQUIREMENT ====
"pytest"
"""

actual = _split_app_and_test_req(response)
print(actual)
expected = ('pydantic, ssl, requests', 'pytest')
assert actual == expected

('pydantic, ssl, requests', 'pytest')


In [ ]:
response = """
==== APP REQUIREMENT ====
"pydantic, ssl, requests"

==== TEST REQUIREMENT ====
"""

actual = _split_app_and_test_req(response)
print(actual)
expected = ('pydantic, ssl, requests', '')
assert actual == expected

('pydantic, ssl, requests', '')


In [ ]:
# | export

def _update_toml_file(output_dir: str, app_req: str, test_req: str) -> None:
    toml_file_path = f"{output_dir}/{TOML_FILE_NAME}"
    toml_contents = read_file_contents(toml_file_path)
    data = toml.loads(toml_contents)
    
    app_reqs = [r.strip() for r in app_req.split(",")]
    test_reqs = [r.strip() for r in test_req.split(",")]
    test_reqs = [r for r in test_reqs if r != "pytest"]
    
    data["project"]["dependencies"] = data["project"]["dependencies"] + app_reqs
    data["project"]["optional-dependencies"]["testing"] = data["project"]["optional-dependencies"]["testing"] + test_reqs
    
    toml_string = toml.dumps(data)
    write_file_contents(toml_file_path, toml_string)

In [ ]:
fixture_requirements = """[build-system]
requires = ["hatchling"]

[project]
name = "app"
version = "0.0.1"

dependencies = [
    "faststream[kafka, docs]>=0.1.5",
]

[project.optional-dependencies]
lint = [
    "black==23.9.1",
]

static-analysis = [
    "types-PyYAML",
]

testing = [
    "faststream[kafka, testing]>=0.1.5",
]

dev = ["app[lint,static-analysis,testing]"]

[tool.pytest.ini_options]
filterwarnings =["ignore::DeprecationWarning"]
"""

app_req = "pydantic, ssl, requests"
test_req = "pytest"

expected = """[build-system]
requires = [ "hatchling",]

[project]
name = "app"
version = "0.0.1"
dependencies = [ "faststream[kafka, docs]>=0.1.5", "pydantic", "ssl", "requests",]

[project.optional-dependencies]
lint = [ "black==23.9.1",]
static-analysis = [ "types-PyYAML",]
testing = [ "faststream[kafka, testing]>=0.1.5",]
dev = [ "app[lint,static-analysis,testing]",]

[tool.pytest.ini_options]
filterwarnings = [ "ignore::DeprecationWarning",]
"""

with TemporaryDirectory() as d:
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(toml_file, fixture_requirements)
    
    _update_toml_file(d, app_req, test_req)
    
    actual = read_file_contents(toml_file)
    print(actual)
    assert actual == expected

[build-system]
requires = [ "hatchling",]

[project]
name = "app"
version = "0.0.1"
dependencies = [ "faststream[kafka, docs]>=0.1.5", "pydantic", "ssl", "requests",]

[project.optional-dependencies]
lint = [ "black==23.9.1",]
static-analysis = [ "types-PyYAML",]
testing = [ "faststream[kafka, testing]>=0.1.5",]
dev = [ "app[lint,static-analysis,testing]",]

[tool.pytest.ini_options]
filterwarnings = [ "ignore::DeprecationWarning",]



In [ ]:
# | export


def _validate_response(
    response: str, output_directory: str, **kwargs: Dict[str, Any]
) -> List[str]:
    try:
        app_req, test_req = _split_app_and_test_req(response)
    except (IndexError, ValueError) as e:
        return [
            "Please add ==== APP REQUIREMENT ==== and ==== TEST REQUIREMENT ==== in your response"
        ]
    
    _update_toml_file(output_directory, app_req, test_req)
    
    return _setup_venv_and_run_tests(output_directory)

In [ ]:
fixture_response = """
"pydantic, ssl, requests"

==== TEST REQUIREMENT ====
"pytest"
"""

with TemporaryDirectory() as d:
    expected = ['Please add ==== APP REQUIREMENT ==== and ==== TEST REQUIREMENT ==== in your response']
    actual = _validate_response(fixture_response, d)
    print(actual)
    assert actual == expected

['Please add ==== APP REQUIREMENT ==== and ==== TEST REQUIREMENT ==== in your response']


In [ ]:
fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
def test_always_pass():
    assert True
"""

fixture_response = """
==== APP REQUIREMENT ====
"pydantic"

==== TEST REQUIREMENT ====
"pytest"
"""


with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_requirements)
    
    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()
    
    expected = []
    actual = _validate_response(fixture_response, d)
    print(actual)
    assert actual == expected

[]


In [ ]:
fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
def test_always_fail():
    assert False
"""

fixture_response = """
==== APP REQUIREMENT ====
"pydantic"

==== TEST REQUIREMENT ====
"pytest"
"""


with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_requirements)
    
    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()
    
    actual = _validate_response(fixture_response, d)
    print(actual[0])
    assert "=================================== FAILURES ===================================" in actual[0]
    print("OK")

============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.2, pluggy-1.3.0
rootdir: /tmp/tmpl0aw7_87
configfile: pyproject.toml
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

tests/test_application.py F                                              [100%]

=================================== FAILURES ===================================
_______________________________ test_always_fail _______________________________
tests/test_application.py:3: in test_always_fail
    assert False
E   assert False
=========================== short test summary info ============================
FAILED tests/test_application.py::test_always_fail - assert False
============================== 1 failed in 0.04s ===============================
OK


In [ ]:
# | export


@retry_on_error()  # type: ignore
def _generate(
    model: str,
    prompt: str,
    app_and_test_code: str,
    total_usage: List[Dict[str, int]],
    output_directory: str,
    **kwargs,
) -> Tuple[str, List[Dict[str, int]], bool]:
    requirements_generator = CustomAIChat(
        params={
            "temperature": 0.2,
        },
        model=model,
        user_prompt=prompt,
    )
    requirements_validator = ValidateAndFixResponse(
        requirements_generator, _validate_response
    )
    validator_result = requirements_validator.fix(
        app_and_test_code,
        total_usage,
        STEP_LOG_DIR_NAMES["requirements"],
        str(output_directory),
        **kwargs,
    )
    return (
        (validator_result, True) # type: ignore
        if isinstance(validator_result[-1], defaultdict)
        else validator_result
    )

In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some valid prompt"
app_skeleton = "some app skeleton"
total_usage = []

test_response = """
==== APP REQUIREMENT ====
"pydantic"

==== TEST REQUIREMENT ====
"pytest"
"""

fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
def test_always_fail():
    assert False
"""


with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_requirements)
    
    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()

    with mock_openai_create(test_response):
        total_usage, is_valid_req_code = _generate(
            model, prompt, app_skeleton, total_usage, d
        )
        
    print(is_valid_req_code)
    
    assert not is_valid_req_code
    assert isinstance(is_valid_req_code, bool)

False


In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some valid prompt"
app_skeleton = "some app skeleton"
total_usage = []

test_response = """
==== APP REQUIREMENT ====
"pydantic"

==== TEST REQUIREMENT ====
"pytest"
"""

fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
def test_always_pass():
    assert True
"""

with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_requirements)
    
    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()

    with mock_openai_create(test_response):
        total_usage, is_valid_req_code = _generate(
            model, prompt, app_skeleton, total_usage, d
        )
        
    print(is_valid_req_code)
    
    assert is_valid_req_code
    assert isinstance(is_valid_req_code, bool)

True


In [ ]:
# | export


def fix_requirements_and_run_tests(
    output_directory: str,
    model: str,
    total_usage: List[Dict[str, int]],
) -> Tuple[List[Dict[str, int]], bool]:
    with yaspin(
        text="Running integration tests...", color="cyan", spinner="clock"
    ) as sp:
        app_file = Path(output_directory) / APPLICATION_FILE_PATH
        app_code = read_file_contents(str(app_file))

        test_file = Path(output_directory) / TEST_FILE_PATH
        test_code = read_file_contents(str(test_file))

        app_and_test_code = f"==== APP CODE ====\n\n{app_code}\n\n==== TEST CODE ====\n\n{test_code}\n\n"

        total_usage, is_requirements_file_valid = _generate(
            model,
            REQUIREMENTS_GENERATION_PROMPT,
            app_and_test_code,
            total_usage,
            output_directory,
        )

        sp.text = ""
        if is_requirements_file_valid:
            message = " ✔ Integration tests were successfully completed."
        else:
            message = " ✘ Error: Integration tests failed."
            sp.color = "red"

        sp.ok(message)

        return total_usage, is_requirements_file_valid

In [ ]:
model = OpenAIModel.gpt3.value
total_usage = []

test_response = """
==== APP REQUIREMENT ====
"pydantic"

==== TEST REQUIREMENT ====
"pytest"
"""

fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
def test_always_pass():
    assert True
"""

with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_requirements)

    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()

    with mock_openai_create(test_response):
        total_usage, is_requirements_file_valid = fix_requirements_and_run_tests(d, model, [])

    print(is_requirements_file_valid)

    assert is_requirements_file_valid
    assert isinstance(is_requirements_file_valid, bool)

⠹ Running integration tests... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ Integration tests were successfully completed. 
True


In [ ]:
model = OpenAIModel.gpt3.value
total_usage = []

test_response = """
==== APP REQUIREMENT ====
"pydantic"

==== TEST REQUIREMENT ====
"pytest"
"""

fixture_app_code = """
print("Hi")
"""

fixture_test_code = """
def test_always_pass():
    assert False
"""

with TemporaryDirectory() as d:
    app_file = Path(d) / APPLICATION_FILE_PATH
    test_file = Path(d) / TEST_FILE_PATH
    toml_file = Path(d) / TOML_FILE_NAME
    write_file_contents(app_file, fixture_app_code)
    write_file_contents(test_file, fixture_test_code)
    write_file_contents(toml_file, fixture_requirements)

    test_init_file_path = test_file.parent / "__init__.py"
    test_init_file_path.touch()

    app_init_file_path = app_file.parent / "__init__.py"
    app_init_file_path.touch()

    with mock_openai_create(test_response):
        total_usage, is_requirements_file_valid = fix_requirements_and_run_tests(d, model, [])

    print(is_requirements_file_valid)

    assert not is_requirements_file_valid
    assert isinstance(is_requirements_file_valid, bool)

⠹ Running integration tests... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✘ Error: Integration tests failed. 
False


/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
